In [1]:
!pip install geopandas #
!pip install tfidf_matcher # test this cool lib idea tfidfvec for string similarity /typo buster

     |████████████████████████████████| 1.0MB 24.4MB/s 
     |████████████████████████████████| 15.4MB 171kB/s 
     |████████████████████████████████| 6.6MB 32.6MB/s 


In [2]:
import folium
import pandas as pd
import random

In [306]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}


In [467]:
poi_pos_satpam=[-6.235440, 106.823182]
poi_masjid=[-6.234996, 106.822583]

some_pos_satpam_hist={'lat':[poi_pos_satpam[0]+0.0001*random.uniform(0,1) for i in range(0,5)],
                      'lon':[poi_pos_satpam[1]+0.0001*random.uniform(0,1) for i in range(0,5)],
                      'message':['saya tunggu di pos satpam pengecekan',
                                 'di dekat gerbang check in',
                                 'di check in gate',
                                 'saya di pos tempat check in di bagian selatan gedung wisma mulia',
                                 'di pos cek in']}

some_masjid_hist={'lat':[poi_masjid[0]+0.0001*random.uniform(0,1) for i in range(0,5)],
                      'lon':[poi_masjid[1]+0.0001*random.uniform(0,1) for i in range(0,5)],
                      'message':['saya tunggu di depan masjid',
                                 'di masjid kompleks wisma mulia',
                                 'saya di depan masjid ar rohmah',
                                 'saya tunggu di pinggir jalan dekat masjid deket pertigaan',
                                 'di masjid ar rohmah']}

some_front_building_hist={'lat':[-6.234793+0.0001*random.uniform(0,1) for i in range(0,5)],
                          'lon':[106.823570+0.0001*random.uniform(0,1) for i in range(0,5)],
                          'message':['saya tunggu di depan gedung wisma mulia',
                                 'di lobby depan gedung wisma mulia',
                                 'saya di depan gedung ngab',
                                 'depan gedung dekat jalan utama',
                                 'ini saya udah di depan gedung']}

map = folium.Map(location=[-6.2356108,106.8229201],zoom_start=20)
folium.TileLayer('cartodbdark_matter').add_to(map)

folium.GeoJson('/content/drive/MyDrive/NLP_geocoding/building_A.geojson').add_to(map)
folium.GeoJson('/content/drive/MyDrive/NLP_geocoding/Building_B.geojson').add_to(map)

folium.CircleMarker(
    [-6.235238, 106.823253],
    radius=5,
    popup = ('PIN POINT'),
    color='white',
    fill=True,
    fill_opacity=1
    ).add_to(map)

for lat, lon,msg in zip(some_pos_satpam_hist.get('lat'),some_pos_satpam_hist.get('lon'),some_pos_satpam_hist.get('message')):
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup = ('message: ' + msg),
        color='blue',
        fill=True,
        fill_opacity=1
        ).add_to(map)

for lat, lon,msg in zip(some_masjid_hist.get('lat'),some_masjid_hist.get('lon'),some_masjid_hist.get('message')):
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup = ('message: ' + msg),
        color='red',
        fill=True,
        fill_opacity=1
        ).add_to(map)

for lat, lon,msg in zip(some_front_building_hist.get('lat'),some_front_building_hist.get('lon'),some_front_building_hist.get('message')):
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup = ('message: ' + msg),
        color='green',
        fill=True,
        fill_opacity=1
        ).add_to(map)

df_hist_loc=pd.concat([pd.DataFrame(some_pos_satpam_hist),pd.DataFrame(some_masjid_hist),pd.DataFrame(some_front_building_hist)])

In [468]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import tfidf_matcher as tm

kmeans=KMeans(n_clusters=3,random_state=1)
scale=StandardScaler()
dat_hist_loc_scalled=scale.fit_transform(df_hist_loc[['lat','lon']])
df_hist_loc['cluster']=kmeans.fit_predict(dat_hist_loc_scalled)



dict_centro={'cluster':[],'lat':[],'lon':[]}
c=0
for ll in scale.inverse_transform(kmeans.cluster_centers_):
  dict_centro['cluster'].append(c)
  dict_centro['lat'].append(ll[0])
  dict_centro['lon'].append(ll[1])
  c+=1
df_centro=pd.DataFrame(dict_centro)

In [469]:
for lat, lon,msg in zip(df_centro.lat,df_centro.lon,df_centro.cluster):
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup = ('cluster centroid: ' + str(msg)),
        color='orange',
        fill=True,
        fill_opacity=1
        ).add_to(map)

In [470]:
df_hist_loc

,lat,lon,message,cluster
0,-6.235374,106.823250,saya tunggu di pos satpam pengecekan,0
1,-6.235383,106.823185,di dekat gerbang check in,0
2,-6.235362,106.823247,di check in gate,0
3,-6.235393,106.823265,saya di pos tempat check in di bagian selatan ...,0
4,-6.235417,106.823193,di pos cek in,0
0,-6.234927,106.822626,saya tunggu di depan masjid,2
1,-6.234914,106.822673,di masjid kompleks wisma mulia,2
2,-6.234945,106.822604,saya di depan masjid ar rohmah,2
3,-6.234994,106.822650,saya tunggu di pinggir jalan dekat masjid deke...,2
4,-6.234927,106.822633,di masjid ar rohmah,2


In [471]:
df_centro

,cluster,lat,lon
0,0,-6.235386,106.823228
1,1,-6.234731,106.823639
2,2,-6.234942,106.822637


In [472]:
df_sugest=(df_hist_loc[df_hist_loc.message.isin(tm.matcher(original=['saya di depan gedung bang'],lookup=df_hist_loc.message.values,k_matches=4,ngram_length=3).iloc[:,2:].values.tolist()[0])].cluster.value_counts(normalize=1)*100).head(1).reset_index()
df_sugest.columns=['cluster','confidence']
ll=df_centro.query(f'cluster=={int(df_sugest.cluster.values)}')


In [473]:
df_sugest

,cluster,confidence
0,1,75.0


In [474]:
map

In [475]:
def plot_sugested_location(msg:'saya tungguin di depan gedung bang',df_centro,df_hist_loc):
  map2 = folium.Map(location=[-6.2356108,106.8229201],zoom_start=35)
  # basemaps['Google Maps'].add_to(map2)
  folium.TileLayer('cartodbdark_matter').add_to(map2)

  folium.GeoJson('/content/drive/MyDrive/NLP_geocoding/building_A.geojson').add_to(map2)
  folium.GeoJson('/content/drive/MyDrive/NLP_geocoding/Building_B.geojson').add_to(map2)
  df_sugest=(df_hist_loc[df_hist_loc.message.isin(tm.matcher(original=[msg],lookup=df_hist_loc.message.values,k_matches=5,ngram_length=3).iloc[:,2:].values.tolist()[0])].cluster.value_counts(normalize=1)*100).head(1).reset_index()
  df_sugest.columns=['cluster','confidence']
  ll=df_centro.query(f'cluster=={int(df_sugest.cluster.values)}')
  folium.Circle(
      [float(ll.lat.values), float(ll.lon.values)],
      radius=3,
      popup = ('sugestion confidence: ' + str(int(df_sugest.confidence.values))+'%'),
      color='red',
      fill=True,
      fill_opacity=1
      ).add_to(map2)
  return map2

In [476]:
map

In [456]:
plot_sugested_location('saya di depan gedung pas ini',df_centro,df_hist_loc,basemaps)

In [465]:
plot_sugested_location('gw di masjid bang',df_centro,df_hist_loc)

In [466]:
plot_sugested_location('saya tungguin di pos checkin ya ngab',df_centro,df_hist_loc)